In [1]:
import pandas as pd
from collections import defaultdict
from datetime import date
import xlsxwriter
from glob import glob
import os

In [2]:
name_dict = {"Yuemei Ding":["yuemei ding","yuemei","ding","ding yuemei","yuemeiding","dingyuemei"],
             "Licong He":["licong he","licong","he","he licong","licong ha","thong ha","le ha","ha","liconghe","helicong","licongha",
                          "thongha","leha","thong le ha","thong li ha","le chong"],
             "Yu Chen":["yu chen","yu","chen yu","yuchen","chenyu"],
             "Yinghua Zhang":["yinghua zhang","yinghua","zhang","zhang yinghua","yinghuazhang","zhangyinghua",'yh'],
             "YT Shih":["yt shih","yt","shih","shih yt","ytshih","shihyt","Amy Shih","amy shih"],
             "Yongyu Zhao":["yongyu zhao","yongyu","zhao","zhao yongyu","yongyuzhao","zhaoyongyu","perry"],
             "Hong Zeng":["hong zeng","hong","zeng","zeng hong","hongzeng","zenghong"],
             "Xiao Chen":["xiao chen","xiao","chen xiao","xiaochen","chenxiao"],
             "Virginia":["virginia"],
             "Chen Ni":["chen ni","ni","ni chen","chenni","nichen"]}

In [3]:
file_path = r"C:\Users\carol\Downloads\81376463214 - Attendee Report.csv"
with open(file_path, 'r', encoding="utf-8") as file: 
    for i, line in enumerate(file):
        if line.startswith("Attendee Details"):
            breakline = i #line index where to break the csvfile
df_original = pd.read_csv(file_path, header=breakline+1, index_col=False)
df_original = df_original[["Attended","User Name (Original Name)","Join Time","Leave Time", "Time in Session (minutes)", 
         "Inviter", "Inviter's SMD"]]
print(f"Total {df_original.shape[0]} attendees. The first 5 are showing below:")
df_original.head()

Total 213 attendees. The first 5 are showing below:


,Attended,User Name (Original Name),Join Time,Leave Time,Time in Session (minutes),Inviter,Inviter's SMD
0,Yes,Yanfang Huang,"Mar 13, 2021 13:14:23","Mar 13, 2021 14:24:42",71,何丽聪,何丽聪
1,Yes,Huaibo Zhang,"Mar 13, 2021 13:03:59","Mar 13, 2021 13:10:36",7,Mike Feng,Xiao Chen
2,Yes,Huaibo Zhang,"Mar 13, 2021 13:10:29","Mar 13, 2021 13:28:03",18,NaN,NaN
3,Yes,Huaibo Zhang,"Mar 13, 2021 13:28:03","Mar 13, 2021 14:34:53",67,NaN,NaN
4,Yes,Yiding Zeng,"Mar 13, 2021 12:58:47","Mar 13, 2021 13:50:10",52,Perry,Zhao


In [4]:
print("Below are rows whose Upline SMD entry is NaN.")
display(df_original[df_original["Inviter's SMD"].isna()])
na_index = df_original[df_original["Inviter's SMD"].isna()].index

Below are rows whose Upline SMD entry is NaN.


,Attended,User Name (Original Name),Join Time,Leave Time,Time in Session (minutes),Inviter,Inviter's SMD
2,Yes,Huaibo Zhang,"Mar 13, 2021 13:10:29","Mar 13, 2021 13:28:03",18,NaN,NaN
3,Yes,Huaibo Zhang,"Mar 13, 2021 13:28:03","Mar 13, 2021 14:34:53",67,NaN,NaN
5,Yes,Yiding Zeng,"Mar 13, 2021 13:50:10","Mar 13, 2021 14:36:40",47,NaN,NaN
10,Yes,Dihua Li,"Mar 13, 2021 14:23:48","Mar 13, 2021 14:23:53",1,NaN,NaN
12,Yes,Anny Gu,"Mar 13, 2021 14:27:56","Mar 13, 2021 14:36:40",9,NaN,NaN
17,Yes,Jingfang Dong,"Mar 13, 2021 13:00:24","Mar 13, 2021 13:52:04",52,NaN,NaN
18,Yes,Jingfang Dong,"Mar 13, 2021 13:52:56","Mar 13, 2021 14:36:40",44,NaN,NaN
20,Yes,Wenting Fan,"Mar 13, 2021 13:42:43","Mar 13, 2021 13:43:46",2,NaN,NaN
26,Yes,Xiao Chen,"Mar 13, 2021 14:30:53","Mar 13, 2021 14:36:19",6,NaN,NaN
28,Yes,Hanna Wu,"Mar 13, 2021 13:43:15","Mar 13, 2021 14:36:41",54,NaN,NaN


In [5]:
df = df_original.dropna(subset=["Inviter's SMD"]) #delete those whose upline smd are NaN

In [6]:
smd_group = defaultdict(list)
for i, row in df.iterrows():
    upline_smd = row["Inviter's SMD"].lower().strip()
    for name in name_dict:
        if upline_smd in name_dict[name]:
            smd_group[name].append(i)

identified_rows = []
for index in smd_group.values():
    identified_rows.extend(index)
print("The following row(s) are not identified:")
display(df.loc[df.index.difference(identified_rows)])
unidentified_index = df.loc[df.index.difference(identified_rows)].index

The following row(s) are not identified:


,Attended,User Name (Original Name),Join Time,Leave Time,Time in Session (minutes),Inviter,Inviter's SMD
0,Yes,Yanfang Huang,"Mar 13, 2021 13:14:23","Mar 13, 2021 14:24:42",71,何丽聪,何丽聪
39,Yes,Maggie Jin,"Mar 13, 2021 12:53:43","Mar 13, 2021 14:36:40",103,Maggie,Yu C hen
53,Yes,Susan Li,"Mar 13, 2021 12:53:28","Mar 13, 2021 14:36:41",104,YT shih,Amy Shih
65,Yes,li 明,"Mar 13, 2021 13:21:39","Mar 13, 2021 13:56:53",36,立明,颖华
82,Yes,zhinuan Yu,"Mar 13, 2021 13:03:15","Mar 13, 2021 14:12:03",69,Yuemei ding,Ben li
94,Yes,Cici Yang,"Mar 13, 2021 12:58:29","Mar 13, 2021 14:36:40",99,Cici Yang,61HUP
111,Yes,YT Deng,"Mar 13, 2021 13:00:29","Mar 13, 2021 14:36:40",97,yuliu,yuliu
145,Yes,Ada A,"Mar 13, 2021 13:42:38","Mar 13, 2021 13:44:10",2,张,张
149,No,Zhijun,--,--,--,YT shih,Amy Shih
151,Yes,Kathy Cai,"Mar 13, 2021 13:00:05","Mar 13, 2021 13:44:38",45,Xiufeng,XiuFeng


In [7]:
other_index = list(na_index) + list(unidentified_index)

In [8]:
print("The groups are displayed below:")
print()
for name, index in smd_group.items():
    print(name)
    display(df.loc[index])

The groups are displayed below:

Xiao Chen


,Attended,User Name (Original Name),Join Time,Leave Time,Time in Session (minutes),Inviter,Inviter's SMD
1,Yes,Huaibo Zhang,"Mar 13, 2021 13:03:59","Mar 13, 2021 13:10:36",7,Mike Feng,Xiao Chen
21,Yes,Ping Yu,"Mar 13, 2021 14:11:30","Mar 13, 2021 14:28:11",17,Mike Feng,Xiao Chen
23,Yes,Sherrie Wang,"Mar 13, 2021 13:42:23","Mar 13, 2021 14:36:12",54,Mike Feng,xiao chen
25,Yes,Xiao Chen,"Mar 13, 2021 12:56:28","Mar 13, 2021 13:50:44",55,Xiao Chen,Xiao Chen
63,No,Gary,--,--,--,Mike Feng,Xiao Chen
93,No,Da,--,--,--,Mike feng,Xiao chen
123,Yes,Qiang Chen,"Mar 13, 2021 13:02:47","Mar 13, 2021 14:25:48",84,Mike Feng,Xiao Chen
128,Yes,Mike Feng,"Mar 13, 2021 12:56:43","Mar 13, 2021 14:36:40",100,Mike Feng,xiao chen
131,No,Jianchen,--,--,--,Mike Feng,Xiao Chen
142,No,Fong,--,--,--,Mike Feng,Xiao Chen


Yongyu Zhao


,Attended,User Name (Original Name),Join Time,Leave Time,Time in Session (minutes),Inviter,Inviter's SMD
4,Yes,Yiding Zeng,"Mar 13, 2021 12:58:47","Mar 13, 2021 13:50:10",52,Perry,Zhao
6,Yes,Aiping Zhang,"Mar 13, 2021 13:00:04","Mar 13, 2021 14:36:40",97,Perry,Zhao
30,Yes,Jianrong Hu,"Mar 13, 2021 12:58:34","Mar 13, 2021 14:34:13",96,Perry,Zhao
43,Yes,Xiaoming Zhao,"Mar 13, 2021 13:10:43","Mar 13, 2021 14:34:03",84,Perry,Zhao
67,Yes,Yongting Liu,"Mar 13, 2021 12:59:07","Mar 13, 2021 14:36:40",98,Perry,Zhao
108,No,Hong,--,--,--,Perry,Zhao
168,Yes,Chunbang Li,"Mar 13, 2021 13:01:07","Mar 13, 2021 14:36:43",96,Perry,Zhao
181,No,Wenbin,--,--,--,Perry,Zhao
195,Yes,May Li,"Mar 13, 2021 13:05:04","Mar 13, 2021 14:36:40",92,Perry,Zhao


Yu Chen


...

,Attended,User Name (Original Name),Join Time,Leave Time,Time in Session (minutes),Inviter,Inviter's SMD
9,Yes,Dihua Li,"Mar 13, 2021 12:58:46","Mar 13, 2021 14:19:26",81,Kelcy,Yuemei Ding
13,Yes,Martin CT,"Mar 13, 2021 13:00:26","Mar 13, 2021 14:00:46",61,sishuo chen,Yuemei Ding
15,Yes,Xiuying Wang,"Mar 13, 2021 13:03:11","Mar 13, 2021 14:34:17",92,Yuemei Ding,Yuemei Ding
19,Yes,Wenting Fan,"Mar 13, 2021 13:00:33","Mar 13, 2021 13:42:42",43,Kelcy,Yuemei Ding
24,Yes,Li Ji,"Mar 13, 2021 13:03:10","Mar 13, 2021 14:36:40",94,Yuemei Ding,Yuemei Ding
...,...,...,...,...,...,...,...
196,Yes,Sishuo Chen,"Mar 13, 2021 13:03:06","Mar 13, 2021 14:36:40",94,Sishuo Chen,Yuemei Ding
199,No,Dr,--,--,--,Hong Hu,Yuemei
202,Yes,Jane Wang,"Mar 13, 2021 14:06:13","Mar 13, 2021 14:36:40",31,Perry,Yuemei Ding
203,No,John,--,--,--,Hong Hu,Yuemei


Yinghua Zhang


,Attended,User Name (Original Name),Join Time,Leave Time,Time in Session (minutes),Inviter,Inviter's SMD
11,Yes,Anny Gu,"Mar 13, 2021 13:13:37","Mar 13, 2021 14:24:47",72,Liming Liu,Yinghua
16,Yes,Jingfang Dong,"Mar 13, 2021 12:59:30","Mar 13, 2021 13:00:06",1,Joyce,Yinghua
42,Yes,Xiaorong Liu,"Mar 13, 2021 12:55:06","Mar 13, 2021 14:36:40",102,Yinghua,Yinghua Zhang
45,Yes,Wei Liu,"Mar 13, 2021 13:24:58","Mar 13, 2021 14:36:40",72,Yinghua,Yinghua
51,No,Shi,--,--,--,Yinghua Zhang,Yinghua
74,No,Chunhua,--,--,--,Yinghua,Yinghua
102,No,Pengyu,--,--,--,Liming Liu,Yinghua zhang
104,Yes,qiang zhou,"Mar 13, 2021 13:01:44","Mar 13, 2021 14:36:40",95,Tiffinany,yinghua zhang
124,No,Yu,--,--,--,Liming Liu,Yinghua Zhang
125,Yes,Kelvin Xia,"Mar 13, 2021 13:15:26","Mar 13, 2021 13:41:55",27,Liming Liu,Yinghua


Licong He


,Attended,User Name (Original Name),Join Time,Leave Time,Time in Session (minutes),Inviter,Inviter's SMD
22,No,Yen,--,--,--,Le ha,Thong Le ha
68,Yes,Niu Hui,"Mar 13, 2021 13:00:40","Mar 13, 2021 14:36:40",96,Licong Ha,Licong Ha
87,Yes,Vicky Tran,"Mar 13, 2021 13:02:48","Mar 13, 2021 13:04:09",2,Mxy20,Le Ha
127,Yes,Jinun He,"Mar 13, 2021 13:58:33","Mar 13, 2021 14:29:55",32,Licong Ha,Licong Ha
147,No,Niu,--,--,--,Licong Ha,Licong Ha
155,Yes,Licong Ha,"Mar 13, 2021 13:08:49","Mar 13, 2021 14:36:40",88,Licong Ha,Licong Ha
192,Yes,Kimling Huynh,"Mar 13, 2021 13:12:30","Mar 13, 2021 14:36:40",85,Licong Ha,Licong Ha


In [9]:
for name, index in smd_group.items():
    file_name = f"{name}_{date.today().strftime('%m-%d-%Y')}_Attendee Report.xlsx"
    workbook = xlsxwriter.Workbook(file_name)
    worksheet = workbook.add_worksheet()
    worksheet.write('A1', 'Attendee Details')
    worksheet.write('A3', 'Attended')
    worksheet.write('B3', 'User Name (Original Name)')
    worksheet.write('C3', 'Join Time')
    worksheet.write('D3', 'Leave Time')
    worksheet.write('E3', 'Time in Session (minutes)')
    worksheet.write('F3', 'Inviter')
    worksheet.write('G3', "Inviter's SMD")
    
    df_group = df.loc[index].reset_index(drop=True)
    for i, row in df_group.iterrows():
        worksheet.write(f'A{i+4}', row['Attended'])
        worksheet.write(f'B{i+4}', row['User Name (Original Name)'])
        worksheet.write(f'C{i+4}', row['Join Time'])
        worksheet.write(f'D{i+4}', row['Leave Time'])
        worksheet.write(f'E{i+4}', row['Time in Session (minutes)'])
        worksheet.write(f'F{i+4}', row['Inviter'])
        worksheet.write(f'G{i+4}', row["Inviter's SMD"])
    workbook.close()

In [10]:
#write others 
df_original.loc[other_index].to_excel(f"Others_{date.today().strftime('%m-%d-%Y')}_Attendee Report.xlsx", index=False)